# Entwurf Lobbytracker

[Dokumentation](https://github.com/knabe-s/lobbytracker_v1/wiki/Dokumentation)

### Graphen initialisieren

In [ ]:
###Import modules
from datetime import datetime
import time
from tqdm import tqdm
import pandas as pd
import networkx as nx

###Using Github
path_gh = r"https://raw.githubusercontent.com/knabe-s/lobbytracker_v1/main/"
path_jp = r"/content/"

###Constructing empty MultiDiGraph (-> multiple directed edges between nodes)
G = nx.MultiDiGraph()
G.graph['name'] = "g_" + str(datetime.now().strftime('%y%m%d-%H%M'))

path_gh_graph = path_gh + G.graph['name'] + "/"

###Adding graph-attributes for generating sequential u_ids
G.graph['count_ids_orig'] = 0
G.graph['count_ids_norm'] = 0
G.graph['count_ids_edges'] = 0

### Befüllen & Updaten des Graphen

####Befüllen [1/2] - Normalformen

In [ ]:
###Loading norm-csv from nextcloud
tmp_files_norm = ["2022-09-20_norm.csv"]

###Adding information about update (date and files) to graph-attributes
tmp_update_norm = "files_norm"
G.graph[tmp_update_norm] = str(tmp_files_norm)

for tmp_files_i in range(len(tmp_files_norm)):
  nc.get_file(path_gh + tmp_files_norm[tmp_files_i])


for tmp_x in range(len(tmp_files_norm)):

  ####Importing data from csv
  tmp_data_file = tmp_files_norm[tmp_x]
  tmp_data_input = pd.read_csv(path_jp + tmp_files_norm[tmp_x])

  tmp_new = False

  for tmp_i in tqdm(range(len(tmp_data_input))):
    ###Generating id for node[i]
    if ("g_" in tmp_data_input.at[tmp_i, "graph_id"]) == False:
      tmp_new = True
      tmp_source = tmp_data_input.at[tmp_i, "aufb_quelle"]
      tmp_date = tmp_data_input.at[tmp_i, "aufb_datum"]
      tmp_ident = "{:07d}".format(int(G.graph['count_ids_norm']) + 1)
      G.graph['count_ids_norm'] = tmp_ident

      tmp_id = "norm_" + tmp_source + "_" + tmp_date + "_#" + str(tmp_ident)
      tmp_data_input.at[tmp_i, "graph_id"] = tmp_id
    else:
      tmp_new = False
      tmp_id = list(G.nodes())[tmp_i]

    ###Constructing attributes for norm-node[i]; checking for each key if value changed
    if tmp_new == False:
      tmp_attr = dict(G.nodes[list(G.nodes())[tmp_i]])
    if tmp_new == True:
      tmp_attr = {}

    tmp_attr['node_typ'] = "norm"
    tmp_attr['node_typ_int'] = 1

    if tmp_new == True:
      tmp_attr['quelle'] = tmp_data_input.at[tmp_i, "aufb_quelle"]
      tmp_attr['datum'] = tmp_data_input.at[tmp_i, "aufb_datum"]
      tmp_attr['bezeichnung'] = tmp_data_input.at[tmp_i, "aufb_bezeichnung"]

    if tmp_new == False: 
      if tmp_attr['quelle'] != tmp_data_input.at[tmp_i, "aufb_quelle"]:
        tmp_attr['quelle'] = tmp_data_input.at[tmp_i, "aufb_quelle"]
      if tmp_attr['datum'] != tmp_data_input.at[tmp_i, "aufb_datum"]:
        tmp_attr['datum'] = tmp_data_input.at[tmp_i, "aufb_datum"]
      if tmp_attr['bezeichnung'] != tmp_data_input.at[tmp_i, "aufb_bezeichnung"]:
        tmp_attr['bezeichnung'] = tmp_data_input.at[tmp_i, "aufb_bezeichnung"]

    ###Adding new norm-node[i] to graph; only if new
    if tmp_new == True:
      G.add_node(tmp_id, **tmp_attr)

  ###Writing filename for new csv with suffix "_nodes"
  tmp_filename_split = tmp_files_norm[tmp_x].split('.')
  tmp_filename = G.graph['name'] + "_" + tmp_filename_split[0] + "_nodes.csv"

  ###End of loop: modified tmp_data_input = tmp_data_output
  tmp_data_output = tmp_data_input

  ###Writing new csv & saving to nextcloud
  for tmp_c in range(len(tmp_files_norm)):
    tmp_data_output.to_csv(path_jp + tmp_filename)
    nc.put_file(path_gh_graph + tmp_filename, path_jp + tmp_filename)

### Resetting temporary variables
%reset_selective -f tmp

100%|██████████| 25388/25388 [00:01<00:00, 23102.98it/s]


####Befüllen [2/2] - Originalformen

In [ ]:
###Loading orig-csv from nextcloud
tmp_files_orig = ["2022-09-20_orig_lobbyisten.csv", "2022-09-20_orig_mitgliedschaften.csv", "2022-09-20_orig_auftraggeber.csv"]

###Adding information about update (date and files) to graph-attributes
tmp_update_orig = "files_orig"
G.graph[tmp_update_orig] = str(tmp_files_orig)

for tmp_files_i in range(len(tmp_files_orig)):
  nc.get_file(path_gh + tmp_files_orig[tmp_files_i])

for tmp_x in range(len(tmp_files_orig)):

  ####Importing data from csv
  tmp_data_file = tmp_files_orig[tmp_x]
  tmp_data_input = pd.read_csv(path_jp + tmp_files_orig[tmp_x])

  for tmp_i in tqdm(range(len(tmp_data_input))):
    ###Constructing empty dictionary for node
    tmp_attr = {}
    tmp_attr['node_typ'] = "orig"
    tmp_attr['node_typ_int'] = 0
    
    ###Generating id for node[i]
    tmp_type = tmp_data_input.at[tmp_i, "aufb_typ"]

    if tmp_type == "lobbyregister_lobbyist":
      tmp_source = tmp_data_input.at[tmp_i, "aufb_quelle"]
      tmp_last_mod = tmp_data_input.at[tmp_i, "aufb_letzte_akt"]
      tmp_ident = tmp_data_input.at[tmp_i, "aufb_quelle_id"]

    elif tmp_type == "lobbyregister_membership":
      tmp_source = tmp_data_input.at[tmp_i, "aufb_quelle"]
      tmp_last_mod = tmp_data_input.at[tmp_i, "aufb_letzte_akt"]
      tmp_ident = "{:07d}".format(int(G.graph['count_ids_orig']) + 1)
      G.graph['count_ids_orig'] = tmp_ident

    elif tmp_type == "lobbyregister_client":
      tmp_source = tmp_data_input.at[tmp_i, "aufb_quelle"]
      tmp_last_mod = tmp_data_input.at[tmp_i, "aufb_letzte_akt"]
      tmp_ident = "{:07d}".format(int(G.graph['count_ids_orig']) + 1)
      G.graph['count_ids_orig'] = tmp_ident

    tmp_id = "orig_" + tmp_source + "_" + tmp_last_mod + "_#" + str(tmp_ident)
    tmp_data_input.at[tmp_i, "graph_id"] = tmp_id

    ###Constructing attributes for orig-node[i]
    if tmp_type == "lobbyregister_lobbyist":
      tmp_attr['typ'] = tmp_type
      tmp_attr['quelle'] = tmp_source
      tmp_attr['letzte_akt'] = tmp_last_mod
      tmp_attr['quelle_id'] = tmp_ident
      tmp_attr['ersteintrag'] = tmp_data_input.at[tmp_i, "aufb_ersteintrag"]
      tmp_attr['jahres_akt'] = tmp_data_input.at[tmp_i, "aufb_jahres_akt"]
      tmp_attr['bezeichnung'] = tmp_data_input.at[tmp_i, "aufb_bezeichnung"]
      tmp_attr['art_rechtsform'] = tmp_data_input.at[tmp_i, "aufb_art_rechtsform"]
      tmp_attr['taetigkeitskategorie'] = tmp_data_input.at[tmp_i, "aufb_taetigkeitskategorie"]
      tmp_attr['intver_finanz_aufw_jahr_von'] = tmp_data_input.at[tmp_i, "aufb_intver_finanz_aufw_jahr_von"]
      tmp_attr['intver_finanz_aufw_jahr_bis'] = tmp_data_input.at[tmp_i, "aufb_intver_finanz_aufw_jahr_bis"]
      tmp_attr['intver_jahr_start'] = tmp_data_input.at[tmp_i, "aufb_intver_jahr_start"]
      tmp_attr['intver_jahr_ende'] = tmp_data_input.at[tmp_i, "aufb_intver_jahr_ende"]
      tmp_attr['intver_anzahl_besch_min'] = tmp_data_input.at[tmp_i, "aufb_intver_anzahl_besch_min"]
      tmp_attr['intver_anzahl_besch_max'] = tmp_data_input.at[tmp_i, "aufb_intver_anzahl_besch_max"]
      tmp_attr['mitglieder_anzahl_wert'] = tmp_data_input.at[tmp_i, "aufb_mitglieder_anzahl_wert"]
      tmp_attr['mitglieder_zaehlung_datum'] = tmp_data_input.at[tmp_i, "aufb_mitglieder_zaehlung_datum"]

      
      if pd.isnull(tmp_data_input.at[tmp_i, "aufb_intver_liste_beschaeftigte_unmittelbar"]) == False:
        tmp_attr['intver_liste_beschaeftigte_unmittelbar'] = str(tmp_data_input.at[tmp_i, "aufb_intver_liste_beschaeftigte_unmittelbar"])
        tmp_list = tmp_data_input.at[tmp_i, "aufb_intver_liste_beschaeftigte_unmittelbar"].split(',')
        tmp_attr['intver_personen_unmittelbar_anzahl'] = len(tmp_list)
        tmp_data_input.at[tmp_i, 'graph_intver_anzahl_beschaeftigte_unmittelbar']
      else:
        tmp_attr['intver_personen_unmittelbar_anzahl'] = float("NaN")
        tmp_data_input.at[tmp_i, 'graph_intver_anzahl_beschaeftigte_unmittelbar']

      if pd.isnull(tmp_data_input.at[tmp_i, "aufb_mitgliedschaften_liste"]) == False:
        tmp_attr['mitgliedschaften_liste'] = str(tmp_data_input.at[tmp_i, "aufb_mitgliedschaften_liste"])
        tmp_list = tmp_data_input.at[tmp_i, "aufb_mitgliedschaften_liste"].split(',')
        tmp_attr['mitgliedschaften_anzahl'] = len(tmp_list)
        tmp_data_input.at[tmp_i, "graph_mitgliedschaften_anzahl"] = tmp_attr['mitgliedschaften_anzahl']
      else:
        tmp_attr['mitgliedschaften_anzahl'] = float("NaN")
        tmp_data_input.at[tmp_i, "graph_mitgliedschaften_anzahl"] = tmp_attr['mitgliedschaften_anzahl']

      if pd.isnull(tmp_data_input.at[tmp_i, "aufb_auftraggeber_liste"]) == False:
        tmp_attr['auftraggeber_liste'] = str(tmp_data_input.at[tmp_i, "aufb_auftraggeber_liste"])
        tmp_list = tmp_data_input.at[tmp_i, "aufb_auftraggeber_liste"].split(',')
        tmp_attr['auftraggeber_anzahl'] = len(tmp_list)
        tmp_data_input.at[tmp_i, "graph_auftraggeber_anzahl"] = tmp_attr['auftraggeber_anzahl']
      else:
        tmp_attr['auftraggeber_anzahl'] = float("NaN")
        tmp_data_input.at[tmp_i, "graph_auftraggeber_anzahl"] = tmp_attr['auftraggeber_anzahl']

    elif tmp_type == "lobbyregister_membership":
      tmp_attr['typ'] = tmp_type
      tmp_attr['quelle'] = tmp_source
      tmp_attr['letzte_akt'] = tmp_last_mod
      tmp_attr['id'] = tmp_ident
      tmp_attr['bezeichnung'] = tmp_data_input.at[tmp_i, "aufb_bezeichnung"]

    elif tmp_type == "lobbyregister_client":
      tmp_attr['typ'] = tmp_type
      tmp_attr['quelle'] = tmp_source
      tmp_attr['letzte_akt'] = tmp_last_mod
      tmp_attr['id'] = tmp_ident
      tmp_attr['bezeichnung'] = tmp_data_input.at[tmp_i, "aufb_bezeichnung"]
  
    ###Adding new node[i] to graph
    G.add_node(tmp_id, **tmp_attr)

  ###Writing filename for new csv with suffix "_nodes"
  tmp_filename_split = tmp_files_orig[tmp_x].split('.')
  tmp_filename = G.graph['name'] + "_" + tmp_filename_split[0] + "_nodes.csv"

  ###End of loop: modified tmp_data_input = tmp_data_output
  tmp_data_output = tmp_data_input

  ###Writing new csv & saving to nextcloud
  for tmp_c in range(len(tmp_files_orig)):
    tmp_data_output.to_csv(path_jp + tmp_filename)
    nc.put_file(path_gh_graph + tmp_filename, path_jp + tmp_filename)

### Resetting temporary variables
%reset_selective -f tmp

100%|██████████| 1463/1463 [00:00<00:00, 28405.75it/s]


###Generieren der Kanten

In [ ]:
###Import of modified orig_nodes.csvs (added norm-ids)
tmp_files_orig = G.graph['files_orig'].replace("[", "")
tmp_files_orig = tmp_files_orig.replace("]", "")
tmp_files_orig = tmp_files_orig.replace("'", "")
tmp_files_orig = tmp_files_orig.split(", ")
tmp_filepath = []

for tmp_files_i in range(len(tmp_files_orig)):
  tmp_list = tmp_files_orig[tmp_files_i].split(".")
  tmp_filepath.append(G.graph['name'] + "_" + tmp_list[0] + "_nodes_nf.csv")
  nc.get_file(path_gh_graph + tmp_filepath[tmp_files_i])

for tmp_x in range(len(tmp_files_orig)):
  ####Importing data from csv & adding 
  if "orig_lobbyisten" in tmp_filepath[tmp_x]:
    tmp_data_file_orig_lobbyists = path_jp + tmp_filepath[tmp_x]
    tmp_data_input = pd.read_csv(tmp_data_file_orig_lobbyists)
    for tmp_c in tqdm(range(len(tmp_data_input)), desc="Lobbyist: Norm -> Orig"):
      tmp_edges_source = tmp_data_input.at[tmp_c, 'aufb_normalform_id']
      tmp_edges_target = tmp_data_input.at[tmp_c, 'graph_id']
      tmp_edges_attr = {"edge_typ": "ist die Normalform von"}
      G.add_edge(tmp_edges_source, tmp_edges_target, **tmp_edges_attr)

  if "orig_mitgliedschaften" in tmp_filepath[tmp_x]:
    tmp_data_file_orig_memberships = path_jp + tmp_filepath[tmp_x]
    tmp_data_input = pd.read_csv(tmp_data_file_orig_memberships)
    for tmp_c in tqdm(range(len(tmp_data_input)), desc="Mitgliedschaften: Norm -> Orig"):
      tmp_edges_source = tmp_data_input.at[tmp_c, 'aufb_normalform_id']
      tmp_edges_target = tmp_data_input.at[tmp_c, 'graph_id']
      tmp_edges_attr = {"edge_typ": "ist die Normalform von"}
      G.add_edge(tmp_edges_source, tmp_edges_target, **tmp_edges_attr)

  if "orig_auftraggeber" in tmp_filepath[tmp_x]:
    tmp_data_file_orig_clients = path_jp + tmp_filepath[tmp_x]
    tmp_data_input = pd.read_csv(tmp_data_file_orig_clients)
    for tmp_c in tqdm(range(len(tmp_data_input)), desc="Auftraggeber: Norm -> Orig"):
      tmp_edges_source = tmp_data_input.at[tmp_c, 'aufb_normalform_id']
      tmp_edges_target = tmp_data_input.at[tmp_c, 'graph_id']
      tmp_edges_attr = {"edge_typ": "ist die Normalform von"}
      G.add_edge(tmp_edges_source, tmp_edges_target, **tmp_edges_attr) 

###Edges orig_client_id -> orig_lobbyist_id
tmp_data_input_clients = pd.read_csv(tmp_data_file_orig_clients)
tmp_data_input_lobbyists = pd.read_csv(tmp_data_file_orig_lobbyists)

for tmp_i in tqdm(range(len(tmp_data_input_clients)), desc="Client -> Lobbyist"):
  tmp_client = {"client_id": tmp_data_input_clients.at[tmp_i, "graph_id"], "client_bez": tmp_data_input_clients.at[tmp_i, "aufb_bezeichnung"]}
  for tmp_c in range(len(tmp_data_input_lobbyists)):
    tmp_lobbyist = {"lobbyist_id": tmp_data_input_lobbyists.at[tmp_c, "graph_id"], "lobbyist_auftraggeber_liste": tmp_data_input_lobbyists.at[tmp_c, "aufb_auftraggeber_liste"]}
    if tmp_client["client_bez"] in str(tmp_lobbyist["lobbyist_auftraggeber_liste"]):
      tmp_edges_source = tmp_client["client_id"]
      tmp_edges_target = tmp_lobbyist["lobbyist_id"]
      tmp_edges_attr = {"edge_typ": "ist ein Auftraggeber von"}
      G.add_edge(tmp_edges_source, tmp_edges_target, **tmp_edges_attr)
  
###Edges orig_membership_id -> orig_lobbyist_id
tmp_data_input_memberships = pd.read_csv(tmp_data_file_orig_memberships)
tmp_data_input_lobbyists = pd.read_csv(tmp_data_file_orig_lobbyists)

for tmp_i in tqdm(range(len(tmp_data_input_memberships)), desc="Lobbyist -> Membership"):
  tmp_member = {"member_id": tmp_data_input_memberships.at[tmp_i, "graph_id"], "member_bez": tmp_data_input_memberships.at[tmp_i, "aufb_bezeichnung"]}
  for tmp_c in range(len(tmp_data_input_lobbyists)):
    tmp_lobbyist = {"lobbyist_id": tmp_data_input_lobbyists.at[tmp_c, "graph_id"], "lobbyist_mitgliedschaften_liste": tmp_data_input_lobbyists.at[tmp_c, "aufb_mitgliedschaften_liste"]}
    if tmp_member["member_bez"] in str(tmp_lobbyist["lobbyist_mitgliedschaften_liste"]):
      tmp_edges_source = tmp_lobbyist["lobbyist_id"]
      tmp_edges_target = tmp_member["member_id"]
      tmp_edges_attr = {"edge_typ": "ist ein Mitglied in"}
      G.add_edge(tmp_edges_source, tmp_edges_target, **tmp_edges_attr)


### Resetting temporary variables
%reset_selective -f tmp

Lobbyist -> Membership: 100%|██████████| 14185/14185 [11:47<00:00, 20.06it/s]


#### Zusammenfassung des Graphen

In [ ]:
print("Graph: " + G.graph['name'])
print("Importierte Normalformen: " + G.graph['files_norm'])
print("Importierte Originalformen: " + G.graph['files_orig'])
print("-------------")

###Number of nodes:
tmp_nodes = list(G)
tmp_count_nodes_all = len(tmp_nodes)

####Count norm-nodes
tmp_count_nodes_norm, tmp_count_nodes_orig = 0, 0
for tmp_i in range(len(tmp_nodes)):
  if str(tmp_nodes[tmp_i]).startswith("norm") == True:
    tmp_count_nodes_norm = tmp_count_nodes_norm + 1
  elif str(tmp_nodes[tmp_i]).startswith("orig") == True:
    tmp_count_nodes_orig = tmp_count_nodes_orig + 1

tmp_out_count_node = "Knoten: " + str(tmp_count_nodes_all)
tmp_out_count_nodes_norm = "Normalformen (" + str(tmp_count_nodes_norm) + ")"
tmp_out_count_nodes_orig = "Originalformen (" + str(tmp_count_nodes_orig) + ")"

print(tmp_out_count_node + " - " + tmp_out_count_nodes_norm + " & " + tmp_out_count_nodes_orig)

###Number of edges
tmp_edges = G.edges()
tmp_count_edges = len(tmp_edges)

edgelist = nx.to_pandas_edgelist(G)
edgelist.to_csv(path_jp + G.graph['name'] + "_edgelist.csv")
nc.put_file(path_gh_graph + G.graph['name'] + "_edgelist.csv", path_jp + G.graph['name'] + "_edgelist.csv")

edges_values = edgelist['edge_typ'].value_counts()
edges_values_dict = edges_values.to_dict()
print("Kanten: " + str(tmp_count_edges) + " - 'Normalform von' (" + str(edges_values_dict["ist die Normalform von"]) + "), 'Auftraggeber von' (" + str(edges_values_dict["ist ein Auftraggeber von"]) + ") & 'Mitglied in' (" + str(edges_values_dict["ist ein Mitglied in"]) + ")")

###Saving graph as graphml
nx.write_graphml(G, path_jp + G.graph['name'] + '.graphml', encoding='utf-8')
out_graphml = G.graph['name'] + '.graphml'
nc.put_file(path_gh_graph + out_graphml, path_jp + out_graphml)

### Resetting temporary variables
%reset_selective -f tmp

Graph: g_220926-1507
Importierte Normalformen: ['2022-09-20_norm.csv']
Importierte Originalformen: ['2022-09-20_orig_lobbyisten.csv', '2022-09-20_orig_mitgliedschaften.csv', '2022-09-20_orig_auftraggeber.csv']
-------------
Knoten: 46232 - Normalformen (25388) & Originalformen (20844)
Kanten: 77587 - 'Normalform von' (20844), 'Auftraggeber von' (1897) & 'Mitglied in' (54846)


##Abfrage der Daten
https://excalidraw.com/#json=iFW3e9fUdNQD2kXGG7NBK,TUYv3UKMh3yh6ytebnODMg

###Resetting Script

In [ ]:
###Removing all files in jupyter-notebook/content/
!rm -r *

In [ ]:
###Resetting variables
%reset
#%reset_selective G

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
